In [1]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 24.5 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 596 kB 52.9 MB/s 
     |████████████████████████████████| 880 kB 48.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7668092922e84dfcf8eb2332c972dc736fdade5b2f5df98b1a9ba0ef15497810
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
import pandas as pd
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import torch
import pickle
import numpy as np

In [4]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [ ]:
amodel = AutoModelForMaskedLM.from_pretrained("bookpanda/wangchanberta-base-att-spm-uncased-masking")

https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpeyypael4


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

storing https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
creating metadata file for /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
loading configuration file https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
Model config CamembertConfig {
  "_name_or_path": "bookpanda/wangchanberta-base-att-spm-uncased-masking",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_t

Downloading:   0%|          | 0.00/427M [00:00<?, ?B/s]

storing https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
creating metadata file for /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
loading weights file https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
All model checkpoint weights were used when initializing CamembertForMaskedLM.

All the weights of CamembertForMaskedLM were initialized from the model checkpoint at bookpanda/wangchanberta-base-att-s

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))

FILE = "drive/MyDrive/AIBuilders/json/mlm_json_3.pth"
model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [6]:
model

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

# Data Prep

In [7]:
df = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k.pkl', 'rb'))

In [8]:
df_test = df[10576:]
df = df[:10576]
df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.9 * len(df))])

In [11]:
df_train

,input_ids,attention_mask,labels
1284,"[5, 5248, 51, 25004, 25004, 6, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5248, 51, 25101, 265, 6, 1, 1, 1, 1, 1, 1,..."
6997,"[5, 10, 1167, 1045, 878, 25004, 25004, 25004, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[5, 10, 1167, 1045, 878, 627, 70, 265, 6, 1, 1..."
9188,"[5, 10, 1145, 1044, 78, 70, 471, 22167, 19404,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 1145, 1044, 78, 70, 471, 22167, 19404,..."
5076,"[5, 10, 1834, 13725, 2102, 25004, 25004, 199, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[5, 10, 1834, 13725, 2102, 234, 265, 199, 1454..."
1393,"[5, 330, 13621, 25004, 25004, 25004, 6, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 330, 13621, 25105, 265, 265, 6, 1, 1, 1, 1..."
...,...,...,...
4757,"[5, 627, 1280, 25004, 25004, 25004, 25004, 250...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 627, 1280, 231, 265, 265, 726, 265, 126, 1..."
10098,"[5, 729, 25004, 1481, 1116, 6, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 729, 8704, 1481, 1116, 6, 1, 1, 1, 1, 1, 1..."
6320,"[5, 7894, 1063, 145, 1849, 300, 25004, 6, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[5, 7894, 1063, 145, 1849, 300, 73, 6, 1, 1, 1..."
5975,"[5, 2169, 25004, 864, 286, 94, 25004, 25004, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 2169, 15, 864, 286, 94, 415, 260, 23, 695,..."


In [12]:
print(df_train.iloc[0]['input_ids'])
print(df_train.iloc[0]['attention_mask'])
print(df_train.iloc[0]['labels'])

[5, 5248, 51, 25004, 25004, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
df_train = Dataset.from_pandas(df_train)
df_val = Dataset.from_pandas(df_val)
df_test = Dataset.from_pandas(df_test)

In [14]:
df_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
    num_rows: 9518
})

In [15]:
combined_dataset = DatasetDict({
    'train': df_train,
    'test': df_val,
    'valid': df_test})

In [16]:
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 9518
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 1058
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [17]:
torch.cuda.empty_cache()
import gc
del df
del df_train
del df_val
del df_test
gc.collect()

461

In [18]:
from huggingface_hub import notebook_login

notebook_login() # เอาไว้โยนโมเดลขึ้น hugging face ได้เลย

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [19]:
from transformers import TrainingArguments

batch_size = 16
# Show the training loss with every epoch
logging_steps = len(combined_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-masking",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    num_train_epochs = 3,
    fp16=True, # สำหรับคนใช้ GPU
    logging_steps=logging_steps,
)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_dataset["train"],
    eval_dataset=combined_dataset["valid"],
)

Cloning https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking into local empty directory.


Download file pytorch_model.bin:   0%|          | 83.0/427M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

Clean file training_args.bin:  34%|###4      | 1.00k/2.92k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/427M [00:00<?, ?B/s]

Using amp half precision backend


In [23]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: __index_level_0__.
***** Running training *****
  Num examples = 9518
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1785


Epoch,Training Loss,Validation Loss
1,0.019100,0.020692
2,0.019100,0.020692
3,0.019000,0.020692


Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-500
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-500/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-1000
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1000/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-1500
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1500/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1500/pytorch_model.bin
***** Running Evaluat

TrainOutput(global_step=1785, training_loss=0.01904480025506153, metrics={'train_runtime': 2196.4521, 'train_samples_per_second': 13.0, 'train_steps_per_second': 0.813, 'total_flos': 7515096898646016.0, 'train_loss': 0.01904480025506153, 'epoch': 3.0})

In [22]:
FILE = "/content/drive/MyDrive/AIBuilders/json/mlm_json_6.pth"
torch.save(model.state_dict(), FILE)

In [ ]:
trainer.push_to_hub()# โยนขึ้น hugging face

Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking
Configuration saved in wangchanberta-base-att-spm-uncased-masking/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/427M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking
   0ad3488..042aaa7  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking
   042aaa7..d124223  main -> main



'https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/commit/042aaa7a82de0ea8414ca4f8eb1e486c59978c17'

In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    model = model.cuda()
FILE = "drive/MyDrive/AIBuilders/mlm/tagging_nova_75.pth"
loaded_model = BertModel()
loaded_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
loaded_model.eval()

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'lm_head.bias', 'roberta.encoder.layer.1.attention.self.value.bias', 'roberta.encoder.layer.10.attention.self.value.bias', 'roberta.encoder.layer.7.attention.self.query.weight', 'roberta.encoder.layer.11.output.LayerNorm.weight', 'roberta.encoder.layer.10.output.LayerNorm.weight', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.self.value.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.bias', 'roberta.encoder.layer.7.attention.self.key.bias', 'lm_head.dense.bias', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.enco

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    c = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)
        else:
            try:
              label_ids.append(2)
            except:
                label_ids.append(-100)

        previous_word_idx = word_idx
    return label_ids

def evaluate_one_text(model, sentence):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)

    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# if use_cuda:
#     model = model.cuda()

text = "ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ"
ans = []
i_f = evaluate_one_text(loaded_model, text)
print(i_f)
a = tokenizer(text)
b = a['input_ids']
c = tokenizer.convert_ids_to_tokens(b)
print(c)
i_f_len = len(i_f)
for j in range(i_f_len):
  if(i_f[j] == 'i'):
    ph = a['input_ids'][j+1]
    a['input_ids'][j+1] = 25004
    print(tokenizer.decode(a['input_ids']))
    b = {'input_ids': torch.Tensor([a['input_ids']]).type(torch.int64).to(device), 'attention_mask': torch.Tensor([a['attention_mask']]).type(torch.int64).to(device)}
    token_logits = model(**b).logits
    mask_token_index = torch.where(b["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    ans.append((j, top_5_tokens[0]))
    text = ''.join(tokenizer.convert_ids_to_tokens(a['input_ids']))
    for token in top_5_tokens:
        print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")
    a['input_ids'][j+1] = ph

print(a)
for x,y in ans:
  a['input_ids'][x+1] = y
print(''.join(tokenizer.convert_ids_to_tokens(a['input_ids'])))

['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'i', 'i', 'i']
['<s>', '▁', 'ประเทศ', 'เรา', 'ผลิต', 'และ', 'ส่งออก', 'ยา', '▁', 'สูบ', 'เยอะ', 'สุดในโลก', 'จิง', 'ป่าว', 'คับ', '</s>']
<s> ประเทศเราผลิตและส่งออกยา สูบเยอะสุดในโลก<mask>ป่าวคับ</s>
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกหรือป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกอะป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกหรื อป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลก จริง ๆป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกอ๊ะป่าวคับ</s>'
<s> ประเทศเราผลิตและส่งออกยา สูบเยอะสุดในโลกจิง<mask>คับ</s>
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง<pad>คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิงไหมคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง<s>คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง_คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิงอะคับ</s>'
<s> ประเทศเราผลิตและส่งออกยา สูบเ

In [ ]:
asdsf = pd.read_csv("drive/MyDrive/AIBuilders/final.csv")

In [ ]:
for i in range(100,200):
  print(asdsf.iloc[i]['text'])

ต้องลอง555
อยากได้เตาอบบาบีก้อน จะเอามาทำขนมให้ลูกๆ ไปทานเล่นระหว่างวันค่ะ เป็นแม่บ้านว่างๆ ก็จะสรรหาเมนูใหม่ๆ ตามยูทูปเอามาลองทำ มีลูกนี่แหละเป็นหนูทดลอง 555 เห็นเตาก้อนแล้วอยากทำขนมคุ๊กกี้อบมากเลยค่ะ เอาเป็นรูปซานต้ากับต้นคริสมาส ฝึกทันพอดีก่อนเทศกาลเลย ❤️❤️❤️❤️❤️
เกิน 50 ไม่ไป!
แฟชั่นไอส์แลนด์ วันที่3/12ค่า
ลดพิเศษเเต่ของน้อยไม่ค่อยเติม
ฮื้ออออ เค้าสับเพร่า เค้าสับเพร่าาาาา เค้าจ่ายเต็มมมมม แต่อร่อยเด้ออออ ฮื้อออ
ผ่านนิเทศไปก่อนนะ
ไปคนเดียวดีฝ่าา
สาขาเพลินนารี่ ถ้าไม่อยากขายก้อปิดเถอะคับ ดูแลไม่ดีแล้วพนง.ยังไม่มีจิตใจในการขายด้วย เวลา1ชม.กับหลายๆอย่างไม่ไหวคับ หาคนมาดูบ้างนะครับ
ขาดเลยละ55 ไฮเนเก้น พอๆๆ 2 ขวด 555
ลอรีอัล รีไวทัลลิฟท์ เดอร์มาลิฟท์เดย์ครีม 50กรัม/ไนท์ ครีม 50กรัม ได้รับคูปองส่วนลด จากราคาเต็ม 599 บาท ลด 120บาท เหลือ 479บาท ในโปรโมชั่น watsons Beauty in Summer (29 มี.ค.-25 เม.ย. 61)
แล้วโครงการอื่นล่ะ??? แสนสิริเตรียมแจงกรณีปัญหาผนังห้องชุดโครงการเดอะ เบส สุขุมวิท 77 พรุ่งนี้ หลังพบใช้โฟมเป็นวัสดุก่อผนัง”
มีซูชิหรอ
นิสสันโน๊ตครับ
ขอบพระคุณคร้าบ
ขอสำรองได้มั้ยอ่า
ตอนนอน